In [ ]:
!git clone https://github.com/chiayewken/Span-ASTE.git
!cd Span-ASTE && git checkout 7cbf035
!cp -a Span-ASTE/* .
!bash setup.sh
!wget https://github.com/chiayewken/Span-ASTE/releases/download/v1.0.0/pretrained_14lap.tar
!tar -xf pretrained_14lap.tar

Cloning into 'Span-ASTE'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 104 (delta 36), reused 77 (delta 21), pack-reused 0
Receiving objects: 100% (104/104), 519.43 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Note: checking out '7cbf035'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 7cbf035 Add SpanModel with scikit-learn-style methods for easy usage (fit, predict, score)
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 353 kB 38.9 MB/s 
     |████

In [ ]:
#@title Data Exploration
data_name = "14lap" #@param ["14lap", "14res", "15res", "16res"]

import sys
sys.path.append("aste")
from data_utils import Data

path = f"aste/data/triplet_data/{data_name}/train.txt"
data = Data.load_from_full_path(path)

for s in data.sentences[:3]:
    print("tokens:", s.tokens)
    for t in s.triples:
        print("target:", (t.t_start, t.t_end))
        print("opinion:", (t.o_start, t.o_end))
        print("label:", t.label)
    print()

tokens: ['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.']
target: (16, 17)
opinion: (15, 15)
label: LabelEnum.positive

tokens: ['it', 'is', 'of', 'high', 'quality', ',', 'has', 'a', 'killer', 'GUI', ',', 'is', 'extremely', 'stable', ',', 'is', 'highly', 'expandable', ',', 'is', 'bundled', 'with', 'lots', 'of', 'very', 'good', 'applications', ',', 'is', 'easy', 'to', 'use', ',', 'and', 'is', 'absolutely', 'gorgeous', '.']
target: (4, 4)
opinion: (3, 3)
label: LabelEnum.positive
target: (9, 9)
opinion: (8, 8)
label: LabelEnum.positive
target: (26, 26)
opinion: (25, 25)
label: LabelEnum.positive
target: (31, 31)
opinion: (29, 29)
label: LabelEnum.positive

tokens: ['Easy', 'to', 'start', 'up', 'and', 'does', 'not', 'overheat', 'as', 'much', 'as', 'other', 'laptops', '.']
target: (2, 3)
opinion: (0, 0)
label: LabelEnum.positive



In [ ]:
# Use pretrained SpanModel weights for prediction
import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

def predict_sentence(text: str, model: SpanModel) -> Sentence:
    path_in = "temp_in.txt"
    path_out = "temp_out.txt"
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False, weight=1, id=0)
    data = Data(root=Path(), data_split=SplitEnum.test, sentences=[sent])
    data.save_to_path(path_in)
    model.predict(path_in, path_out)
    data = Data.load_from_full_path(path_out)
    return data.sentences[0]

text = "Did not enjoy the new Windows 8 and touchscreen functions ."
model = SpanModel(save_dir="pretrained_14lap", random_seed=0)
sent = predict_sentence(text, model)

for t in sent.triples:
    target = " ".join(sent.tokens[t.t_start:t.t_end+1])
    opinion = " ".join(sent.tokens[t.o_start:t.o_end+1])
    print()
    print(dict(target=target, opinion=opinion, sentiment=t.label))

{'command': 'cd /content && allennlp predict pretrained_14lap/weights/model.tar.gz /content/pretrained_14lap/temp_data/pred_in.json --predictor span_model --include-package span_model --use-dataset-reader  --output-file pretrained_14lap/temp_data/pred_out.json --cuda-device 0 --silent '}
################################################################################
################################################################################
{'locals': ('use_ner_embeds', False)}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_double_mix_embedder', False)}
{'locals': ('relation_head_type', 'proper')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys(['focal_loss_gamma', 'use_bi_affine_pruner', 'use_classify_mask_pruner', 'use_focal_loss', 'use_ner_scores_for_prune', 'use_ope_down_project', 'use_pair_feature_multiply', 'use_pairwise_down_project', 'use_span_loss_for_pruners', 'use_span_pair_aux_task', 'use_span

In [ ]:
# Train SpanModel from scratch
random_seed = 0
path_train = f"aste/data/triplet_data/{data_name}/train.txt"
path_dev = f"aste/data/triplet_data/{data_name}/dev.txt"
path_test = f"aste/data/triplet_data/{data_name}/test.txt"
save_dir = f"outputs/{data_name}/seed_{random_seed}"

model = SpanModel(save_dir=save_dir, random_seed=random_seed)
model.fit(path_train, path_dev)

{'weights_dir': PosixPath('outputs/14lap/seed_0/weights')}
{'random_seed': 0}
{'pytorch_seed': 0}
{'numpy_seed': 0}
{'train_data_path': PosixPath('/content/outputs/14lap/seed_0/temp_data/train.json')}
{'validation_data_path': PosixPath('/content/outputs/14lap/seed_0/temp_data/validation.json')}
{'test_data_path': PosixPath('/content/outputs/14lap/seed_0/temp_data/test.json')}
{'path_config': PosixPath('outputs/14lap/seed_0/config.jsonnet')}
{'command': 'cd /content && allennlp train outputs/14lap/seed_0/config.jsonnet --serialization-dir outputs/14lap/seed_0/weights --include-package span_model'}
2022-04-28 06:11:40,547 - INFO - allennlp.common.params - random_seed = 0
2022-04-28 06:11:40,547 - INFO - allennlp.common.params - numpy_seed = 0
2022-04-28 06:11:40,548 - INFO - allennlp.common.params - pytorch_seed = 0
2022-04-28 06:11:40,552 - INFO - allennlp.common.checks - Pytorch version: 1.7.0
2022-04-28 06:11:40,552 - INFO - allennlp.common.params - type = default
2022-04-28 06:11:40,

In [ ]:
# Evaluate SpanModel F1 Score
import json

path_pred = "pred.txt"
model.predict(path_in=path_test, path_out=path_pred)
results = model.score(path_pred, path_test)
print(json.dumps(results, indent=2))

{'command': 'cd /content && allennlp predict outputs/14lap/seed_0/weights/model.tar.gz /content/outputs/14lap/seed_0/temp_data/pred_in.json --predictor span_model --include-package span_model --use-dataset-reader  --output-file outputs/14lap/seed_0/temp_data/pred_out.json --cuda-device 0 --silent '}
################################################################################
################################################################################
{'locals': ('use_ner_embeds', False)}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_double_mix_embedder', False)}
{'locals': ('relation_head_type', 'proper')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys(['focal_loss_gamma', 'use_bi_affine_pruner', 'use_classify_mask_pruner', 'use_focal_loss', 'use_ner_scores_for_prune', 'use_ope_down_project', 'use_pair_feature_multiply', 'use_pairwise_down_project', 'use_span_loss_for_pruners', 'use_span_pair_aux_task